In [5]:
import os
import json

## Initiate data_setting.js

In [6]:
# list all directories in the current folder
models = [d for d in os.listdir() if os.path.isdir(d)]
d = {e:e for e in models}
s = 'name_to_folder_map = ' + json.dumps(d, indent=2)
with open('data_setting.js', 'w+') as f:
    f.write(s)

## create a data.js file for the explorer

In [7]:
# models = [d for d in os.listdir() if os.path.isdir(d)]
for model in models:
    files = os.listdir(f'./{model}')
    file = [f for f in files if f.startswith("output")][0]
    with open(os.path.join(model, file), 'r') as f:
        obj = json.load(f)
    keys_needed = ['response']
    new_obj = {}
    for i in range(1, 1001):
        if str(i) not in obj: continue
        new_obj[str(i)] = {k:v for k,v in obj[str(i)].items() if k in keys_needed}

    outstr = f"cache['{model}'] = {json.dumps(new_obj, indent=2)}"
    with open(os.path.join(model, 'data.js'), 'w') as f:
        f.write(outstr)


## create a score.js file for the leaderboard

In [8]:
def extract_import_values(obj):
    out = {}
    out['Model'] = ""               # We need Model to be the first column
    out['ALL'] = obj['average']['accuracy']
    out['FQA'] = obj['task']['figure question answering']['accuracy']
    out['GPS'] = obj['task']['geometry problem solving']['accuracy']
    out['MWP'] = obj['task']['math word problem']['accuracy']
    out['TQA'] = obj['task']['textbook question answering']['accuracy']
    out['ALG'] = obj['skills']['algebraic reasoning']['accuracy']
    out['ARI'] = obj['skills']['arithmetic reasoning']['accuracy']
    out['GEO'] = obj['skills']['geometry reasoning']['accuracy']
    out['LOG'] = obj['skills']['logical reasoning']['accuracy']
    out['NUM'] = obj['skills']['numeric commonsense']['accuracy']
    out['SCI'] = obj['skills']['scientific reasoning']['accuracy']
    out['STA'] = obj['skills']['statistical reasoning']['accuracy']
    return out


models = [d for d in os.listdir() if os.path.isdir(d)]
ls = []
for model in models:
    files = os.listdir(f'./{model}')
    file = [f for f in files if f.startswith("scores_")][0]
    with open(os.path.join(model, file), 'r') as f:
        obj = json.load(f)
    out = extract_import_values(obj)
    out['Model'] = model
    ls.append(out)

# print to file
with open('model_scores.js', 'w+') as f:
    f.write("score_table = " + str(ls))